In [5]:
import pgmpy.models
import pgmpy.inference
import numpy as n

model = pgmpy.models.BayesianNetwork([('Burglary', 'Alarm'),
									('Earthquake', 'Alarm'),
									('Alarm', 'JohnCalls'),
									('Alarm', 'MaryCalls')])
# Probability of burglary (True, False)
a_burglary = pgmpy.factors.discrete.TabularCPD('Burglary', 2, [[0.001], [0.999]])
print(a_burglary)
# Probability of earthquake (True, False)
a_earthquake = pgmpy.factors.discrete.TabularCPD('Earthquake', 2, [[0.002], [0.998]])
# Probability of alarm going of (True, False) given a burglary and/or earthquake
a_alarm = pgmpy.factors.discrete.TabularCPD('Alarm', 2, [[0.95, 0.94, 0.29, 0.001],
                                                           [0.05, 0.06, 0.71, 0.999]],
                                              evidence=['Burglary', 'Earthquake'],
                                              evidence_card=[2, 2])
# Probability that John calls (True, False) given that the alarm has sounded
a_john = pgmpy.factors.discrete.TabularCPD('JohnCalls', 2, [[0.90, 0.05],
														   [0.10, 0.95]],
											  evidence=['Alarm'],
											  evidence_card=[2])
# Probability that Mary calls (True, False) given that the alarm has sounded
a_mary = pgmpy.factors.discrete.TabularCPD('MaryCalls', 2, [[0.70, 0.01],
														   [0.30, 0.99]],
											  evidence=['Alarm'],
											  evidence_card=[2])
# Adding
model.add_cpds(a_burglary, a_earthquake, a_alarm, a_john, a_mary)
# Check if the model is valid, throw an exception otherwise
model.check_model()

# Print probability distributions
print('Probability distribution, P(Burglary)')
print(a_burglary)
print()
print('Probability distribution, P(Earthquake)')
print(a_earthquake)
print()
print('Joint probability distribution, P(Alarm | Burglary, Earthquake)')
print(a_alarm)
print()
print('Joint probability distribution, P(JohnCalls | Alarm)')
print(a_john)
print()
print('Joint probability distribution, P(MaryCalls | Alarm)')
print(a_mary)
print()


infer = pgmpy.inference.VariableElimination(model)

posterior_probability = infer.query(['Burglary'], evidence={'JohnCalls': 0, 'MaryCalls': 0})

print('Posterior probability of Burglary if JohnCalls(True) and MaryCalls(True)')
print(posterior_probability)
print()

posterior_probability = infer.query(['Alarm'], evidence={'Burglary': 0, 'Earthquake': 0})

print('Posterior probability of Alarm sounding if Burglary(True) and Earthquake(True)')
print(posterior_probability)
print()

+-------------+-------+
| Burglary(0) | 0.001 |
+-------------+-------+
| Burglary(1) | 0.999 |
+-------------+-------+
Probability distribution, P(Burglary)
+-------------+-------+
| Burglary(0) | 0.001 |
+-------------+-------+
| Burglary(1) | 0.999 |
+-------------+-------+

Probability distribution, P(Earthquake)
+---------------+-------+
| Earthquake(0) | 0.002 |
+---------------+-------+
| Earthquake(1) | 0.998 |
+---------------+-------+

Joint probability distribution, P(Alarm | Burglary, Earthquake)
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.95          | 0.94          | 0.29          | 0.001         |
+-----------

Eliminating: Alarm: 100%|██████████| 2/2 [00:00<00:00, 1055.97it/s]


Posterior probability of Burglary if JohnCalls(True) and MaryCalls(True)
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.2842 |
+-------------+-----------------+
| Burglary(1) |          0.7158 |
+-------------+-----------------+



Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

Posterior probability of Alarm sounding if Burglary(True) and Earthquake(True)
+----------+--------------+
| Alarm    |   phi(Alarm) |
+==========+==============+
| Alarm(0) |       0.9500 |
+----------+--------------+
| Alarm(1) |       0.0500 |
+----------+--------------+

